### Time travel
The goal of this notebook is to add the ability to select data based on how long it has existed.

We'd ideally like to do this in two ways - either by selecting the data release version or by selecting the amount of time between the exposure and now. This is useful if you want to only send data to Zooniverse that is not proprietary (greater than 2 years old).

Here's the forum that talks about how to get MJD info from the Rubin source catalog:
https://community.lsst.org/t/how-should-one-efficiently-get-the-mjd-for-a-source-catalog/3733/4

In [2]:
# Generic python packages
import pylab as plt
import gc

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst
from datetime import datetime
from astropy.time import Time

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

In [3]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collections)

In [4]:
datasetType = 'calexp'
dataId = {'visit': 192350, 'detector': 175}
calexp = butler.get(datasetType, dataId=dataId)

In [5]:
# This is the time of the observation in MJD:
t1 = calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD)
print(t1)
# You can also get UTC:
#print(calexp.getInfo().getVisitInfo().getDate().get(lsst.daf.base.DateTime.MJD,lsst.daf.base.DateTime.UTC))


59837.38009422338


### Below, finding the difference between 'now' and the observation:
To do: Is there a way to improve this (take away the 2*365 option)?

In [6]:
# Now get the time of right now and see their difference:

nt = Time.now()
#ut = Time(datetime.utcnow())



if nt.mjd - t1 > 2*365:
    print(f'Obs is older than 2 years {round(nt.mjd - t1,1)} days old')
else:
    print(f'Obs is younger than 2 years {round(nt.mjd - t1,1)} days old')

Obs is younger than 2 years 63.3 days old


In [7]:
print(nt.mjd - 2*365)

59170.66263853213


### Is there a way to modify the below query to include a time restriction?


In [8]:
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

import lsst.daf.butler as dafButler
import lsst.geom
import lsst.afw.display as afwDisplay

plt.style.use('tableau-colorblind10')
%matplotlib inline

import warnings
from astropy.units import UnitsWarning
import pandas

#initializing Tap and Butler
pandas.set_option('display.max_rows', 20)
warnings.simplefilter("ignore", category=UnitsWarning)
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

In [9]:
max_rec=10 # make 100 for full subject set test
use_center_coords = "62, -37"
use_radius = "1.0"

In [10]:
### Modify the query

In [18]:
# This was the original query from the Citizen_Science_Testing.ipynb tutorial
# also, from the tutorial notebooks from RSP
# I've modified it to have obj.whatever
query = "SELECT TOP " + str(max_rec) + " " + \
        "obj.objectId, obj.coord_ra, obj.coord_dec, obj.detect_isPrimary " + \
        "g_cModelFlux, r_cModelFlux, obj.r_extendedness, obj.r_inputCount " + \
        "FROM dp02_dc2_catalogs.Object as obj " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND r_extendedness = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 18.0 " + \
        "ORDER by r_cModelFlux DESC"
results = service.search(query)
print(results)

<Table length=10>
      objectId       coord_ra   coord_dec  ... r_extendedness r_inputCount
                       deg         deg     ...                            
       int64         float64     float64   ...    float64        int32    
------------------- ---------- ----------- ... -------------- ------------
1567965153859768169 61.6998044 -37.2739044 ...            1.0          106
1650947495431285770 61.1583878 -36.3657651 ...            1.0          105
1651448872733547971 62.5770683 -36.1977335 ...            1.0          116
1651536833663756158 61.8484527 -36.1061415 ...            1.0          105
1651325727431231924 62.7137842 -36.6836861 ...            1.0          117
1651334523524249766 62.3059651 -36.7486842 ...            1.0          110
1650894718873158112 60.8890681 -36.5646567 ...            1.0          107
1651466464919580824 62.1611157  -36.333154 ...            1.0          107
1651325727431231922 62.6921133 -36.6985965 ...            1.0          112
1651528

In [27]:
# Now, this is an example from RSP that uses the forcedsource catalog, the ccdvisit
# catalog, and the objectId catalog.
# These three are necessary because forcedsource contains the matching
# keywords (ccdvisitID and objectId) in order to join ccdvisit with 
# the objectId.
# ccdvisit catalog has info about the time of the exposure.

# This is the direct example from the RSP:
'''
query = "SELECT fs.forcedSourceId, fs.objectId, fs.ccdVisitId, fs.detect_isPrimary, " + \
        "fs.band, scisql_nanojanskyToAbMag(fs.psfFlux) as psfMag, ccd.obsStartMJD, " + \
        "scisql_nanojanskyToAbMag(obj.r_psfFlux) as obj_rpsfMag " + \
        "FROM dp02_dc2_catalogs.ForcedSource as fs " + \
        "JOIN dp02_dc2_catalogs.CcdVisit as ccd " + \
        "ON fs.ccdVisitId = ccd.ccdVisitId " + \
        "JOIN dp02_dc2_catalogs.Object as obj " + \
        "ON fs.objectId = obj.objectId " + \
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 " + \
        "AND obj.r_extendedness = 0 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) > 17.5 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) < 18.0 " + \
        "AND fs.band = 'r' "
'''
# This is my query, with the startmjd requirement
# here, I'm using 50 days before present 'now time' mjd
# Eventually, we'll want to use 2*365 days, but 
# currently there aren't that many old sources
# that meet the requirements
query = "SELECT fs.forcedSourceId, fs.objectId, fs.ccdVisitId, fs.detect_isPrimary, " + \
        "fs.band, scisql_nanojanskyToAbMag(fs.psfFlux) as psfMag, ccd.obsStartMJD, ccd.obsStart " + \
        "FROM dp02_dc2_catalogs.ForcedSource as fs " + \
        "JOIN dp02_dc2_catalogs.CcdVisit as ccd " + \
        "ON fs.ccdVisitId = ccd.ccdVisitId " + \
        "JOIN dp02_dc2_catalogs.Object as obj " + \
        "ON fs.objectId = obj.objectId " + \
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 " + \
        "AND obj.r_extendedness = 0 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) > 17.5 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) < 18.0 " + \
        "AND fs.band = 'r' " + \
        f"AND ccd.obsStartMJD < {nt.mjd - 50} "
results = service.search(query)
print(results)

# Link to table info for ccdvisit/others: https://dm.lsst.org/sdm_schemas/browser/dp02.html#CcdVisit 

<Table length=2909>
  forcedSourceId         objectId      ...          obsStart         
      int64               int64        ...           object          
------------------ ------------------- ... --------------------------
104088585568802545 1567921173394650080 ... 2022-09-18 06:18:29.774199
103716499667057546 1567921173394650080 ...   2022-09-17 08:28:10.2671
104071441133114408 1567921173394650080 ...   2022-09-18 05:55:38.2609
104047259930363137 1567921173394650080 ...   2022-09-18 05:21:18.2258
 21649872503473461 1567921173394650080 ...   2022-02-24 02:04:30.7151
103714346277831446 1567921173394650080 ...   2022-09-17 08:25:31.7231
104064947679422127 1567921173394650080 ...   2022-09-18 05:47:45.2209
103695564922713720 1567921173394650080 ...   2022-09-17 08:01:27.5471
 21650458229624109 1567921173394650080 ...   2022-02-24 02:05:09.5089
103714346277822435 1567859600743513720 ...   2022-09-17 08:25:31.7231
               ...                 ... ...                        ...


# Things to consider

## 1. Are there other options for time in ccdvisit? 
Obs end? There's also midpoint: expMidptMJD. How precise do we want to be?

## 2. Do we care about time zones? 
I'm not sure if we need to convert between Chile and here. Do we care about this type of inaccuracy?
Answer: everything is already in TAI so we don't need to worry.

## 3. Do we want a better way to do this in terms of UTC years?
Instead of number of days (mjd), where we would have to account for leap years.
I can do this, but it's going to be more complicated to do in terms of years: https://stackoverflow.com/questions/765797/convert-timedelta-to-years

In [35]:
# Another way to do this is to use tai times:

print(nt.tai)
from datetime import timedelta
delta = timedelta(days = 365*2)
earliest = nt.tai - delta
print('earliest', earliest)

2022-11-17 15:54:48.969176
earliest 2020-11-17 15:54:48.969176


In [36]:
# Now, this is an example from RSP that uses the forcedsource catalog, the ccdvisit
# catalog, and the objectId catalog.
# These three are necessary because forcedsource contains the matching
# keywords (ccdvisitID and objectId) in order to join ccdvisit with 
# the objectId.
# ccdvisit catalog has info about the time of the exposure.

# This is the direct example from the RSP:
'''
query = "SELECT fs.forcedSourceId, fs.objectId, fs.ccdVisitId, fs.detect_isPrimary, " + \
        "fs.band, scisql_nanojanskyToAbMag(fs.psfFlux) as psfMag, ccd.obsStartMJD, " + \
        "scisql_nanojanskyToAbMag(obj.r_psfFlux) as obj_rpsfMag " + \
        "FROM dp02_dc2_catalogs.ForcedSource as fs " + \
        "JOIN dp02_dc2_catalogs.CcdVisit as ccd " + \
        "ON fs.ccdVisitId = ccd.ccdVisitId " + \
        "JOIN dp02_dc2_catalogs.Object as obj " + \
        "ON fs.objectId = obj.objectId " + \
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 " + \
        "AND obj.r_extendedness = 0 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) > 17.5 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) < 18.0 " + \
        "AND fs.band = 'r' "
'''
# This is my query, with the startmjd requirement
# here, I'm using 50 days before present 'now time' mjd
# Eventually, we'll want to use 2*365 days, but 
# currently there aren't that many old sources
# that meet the requirements
query = "SELECT fs.forcedSourceId, fs.objectId, fs.ccdVisitId, fs.detect_isPrimary, " + \
        "fs.band, scisql_nanojanskyToAbMag(fs.psfFlux) as psfMag, ccd.obsStartMJD, ccd.obsStart " + \
        "FROM dp02_dc2_catalogs.ForcedSource as fs " + \
        "JOIN dp02_dc2_catalogs.CcdVisit as ccd " + \
        "ON fs.ccdVisitId = ccd.ccdVisitId " + \
        "JOIN dp02_dc2_catalogs.Object as obj " + \
        "ON fs.objectId = obj.objectId " + \
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 " + \
        "AND obj.r_extendedness = 0 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) > 17.5 " + \
        "AND scisql_nanojanskyToAbMag(obj.r_cModelFlux) < 18.0 " + \
        "AND fs.band = 'r' " + \
        f"AND ccd.obsStart < {earliest} "
results = service.search(query)
print(results)

# Link to table info for ccdvisit/others: https://dm.lsst.org/sdm_schemas/browser/dp02.html#CcdVisit 

DALQueryError: ADQL syntax error: Encountered " <S_INTEGER> "15 "" at line 1, column 650.
Was expecting one of:
    <EOF> 
    "GROUP" ...
    "LIMIT" ...
    "ORDER" ...
    "OFFSET" ...
    "HAVING" ...
    ";" ...
    "GROUP" ...
    "HAVING" ...
    "ORDER" ...
    "LIMIT" ...
    "OFFSET" ...
    "LIMIT" ...

2022-11-17 15:54:48.969176
earliest 2020-11-17 15:54:48.969176


In [16]:
nt.mjd - 50

59850.66263853213

In [ ]:
# I'm guessing we can add this as a separate util and then incorporate it into the 
# rest of the exammples, so I'm not going to add the sending to Zooniverse stuff here.